In [1]:
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd

In [95]:
gdf = gpd.read_file("../../../all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA.json")
gdf = gdf[gdf["State"] == 'tx']

In [96]:
df = pd.read_csv("TX_2016_Congressional.csv")
df.head(1)

,County,FIPS,VTD,cntyvtd,Office,Name,Party,Incumbent,Votes
0,Anderson,1,0001,10001,President,Clinton,D,N,262


In [97]:
gdf.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
2865,Anderson,1,262,742,18,5,3,1030,tx,tx-anderson-1,...,2651,1965,393,16,37,239,None,None,None,"POLYGON ((-95.60102 31.74519, -95.60095 31.745..."


In [98]:
tx_cong_vote = df[df["Office"].str.contains("U.S. Rep")]

In [99]:
tx_cong_vote.head(1)

,County,FIPS,VTD,cntyvtd,Office,Name,Party,Incumbent,Votes
110,Anderson,1,0001,10001,U.S. Rep 5,Hensarling,R,Y,793


# clean tx_cong_vote

In [100]:
rename = {"Office":"dist", "VTD":"Precinct #"}
tx_cong_vote.rename(rename, axis=1, inplace=True)
tx_cong_vote.drop(columns=["FIPS", "Incumbent"], axis=1, inplace=True)
tx_cong_vote.head(1)

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
110,Anderson,0001,10001,U.S. Rep 5,Hensarling,R,793


In [101]:
def normalizeDist(df):
    for index, row in df.iterrows():
        d = row["dist"]
        d = d.split(" ")[-1]
        df.at[index, "dist"] = d

In [102]:
normalizeDist(tx_cong_vote)

In [103]:
def normalizePrecNum(df):
    for index, row in df.iterrows():
        df.at[index, "Precinct #"] = row["Precinct #"].lstrip("0")
normalizePrecNum(tx_cong_vote)


In [104]:
tx_cong_vote.sort_values(by=["dist", "Precinct #"], inplace=True)
tx_cong_vote.head(1)

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
1170,Angelina,1,50001,1,Gohmert,R,162


In [105]:
grouped = tx_cong_vote.groupby(["dist", "County", "Precinct #", "Name", "Party"])
grouped.sum()

Votes
dist County   Precinct # Name     Party       
1    Angelina 1          Culler   W          0
                         Gohmert  R        162
                         Gray     L          3
                         McKellar D        106
              10A        Culler   W          0
...                                        ...
9    Harris   986        Martin   R         25
              994        Green    D         47
                         Martin   R         20
              995        Green    D        830
                         Martin   R         85

[25967 rows x 1 columns]

In [106]:
# new dataframe
columns = ["dist", "County", "Precinct #", "Cong16Dem",
           "Cong16Rep", "Cong16Lib", "Cong16Grn",
           "Cong16Other", "Cong16Total"]
new_tx_cong = pd.DataFrame(columns=columns)
new_tx_cong.head()

,dist,County,Precinct #,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total


In [107]:
vote_column_name = {'D': "Cong16Dem", 'L': "Cong16Lib", 'G': "Cong16Grn", 'R': "Cong16Rep", 'W': "Cong16Other"}


# iterate the groupby df
print("################## Merging TX Dataframe ##################")

new_row = ""
cur_precinct = ""
total_votes = 0

for index, group in grouped:
    cur_row = group.loc[:].head(1)
    precinct = cur_row["Precinct #"].values[0]

    if precinct != cur_precinct:
        if cur_precinct != "":
            # add TotalVotes to new row
            new_row["Cong16Total"] = total_votes
            # add new_row to new_RI dataframe
            new_tx_cong = pd.concat([new_tx_cong, new_row], ignore_index=True, sort=False)
            
        # reset vars
        cur_precinct = precinct
        new_row = ""
        total_votes = 0
    
    if type(new_row) == str:
        # get columns other than Total (Vote)
        new_row = cur_row[["County", "dist", "Precinct #"]]

    candidate_key = cur_row["Party"].values[0]
    new_row[vote_column_name[candidate_key]] = cur_row["Votes"].values[0]
    total_votes += cur_row["Votes"].values[0]
    
print("\n################ Done Merging TX Dataframe ###############")

################## Merging TX Dataframe ##################

################ Done Merging TX Dataframe ###############


In [108]:
new_tx_cong.head()

,dist,County,Precinct #,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total
0,1,Angelina,1,106,162,3,NaN,0,271
1,1,Angelina,10A,261,352,10,NaN,0,623
2,1,Angelina,10B,31,43,1,NaN,0,75
3,1,Angelina,11,115,1165,17,NaN,1,1298
4,1,Angelina,11B,104,561,17,NaN,1,683


In [109]:
new_tx_cong.isna().sum()

dist              0
County            0
Precinct #        0
Cong16Dem      2197
Cong16Rep       433
Cong16Lib      2621
Cong16Grn      4488
Cong16Other    8581
Cong16Total       0
dtype: int64

In [110]:
a = sorted(list(tx_cong_vote["Precinct #"].unique()))

In [111]:
b = sorted(list(new_tx_cong["Precinct #"].unique()))

In [112]:
for tup in list(zip(a, b)):
    if tup[0] != tup[1]:
        print(tup)

('995', '996')
('996', '997')
('997', '999')
('999', '9N')
('9N', '9S')


In [113]:
# new missing 996
tx_cong_vote[tx_cong_vote["Precinct #"] == "995"]

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
155177,Harris,995,2010995,9,Green,D,830
156185,Harris,995,2010995,9,Martin,R,85


In [114]:
new_tx_cong = new_tx_cong.append({"County":"Harris", "Precinct #":"995", "Cong16Rep":85, "Cong16Dem":830}, 
                                 ignore_index=True)
new_tx_cong

,dist,County,Precinct #,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total
0,1,Angelina,1,106,162,3,NaN,0,271
1,1,Angelina,10A,261,352,10,NaN,0,623
2,1,Angelina,10B,31,43,1,NaN,0,75
3,1,Angelina,11,115,1165,17,NaN,1,1298
4,1,Angelina,11B,104,561,17,NaN,1,683
...,...,...,...,...,...,...,...,...,...
8853,9,Harris,984,123,8,NaN,NaN,NaN,131
8854,9,Harris,985,155,113,NaN,NaN,NaN,268
8855,9,Harris,986,199,25,NaN,NaN,NaN,224
8856,9,Harris,994,47,20,NaN,NaN,NaN,67


In [115]:
new_tx_cong.at[8857, "dist"] = "9"
new_tx_cong.at[8857, "Cong16Total"] = 915

In [116]:
# fill in nulls
new_tx_cong.fillna(value=0, inplace=True)
new_tx_cong.isna().sum()

dist           0
County         0
Precinct #     0
Cong16Dem      0
Cong16Rep      0
Cong16Lib      0
Cong16Grn      0
Cong16Other    0
Cong16Total    0
dtype: int64

In [117]:
add = ["Fort Bend", "De Witt", "El Paso", "Jeff Davis", "Jim Hogg", "Jim Wells", "La Salle", "Live Oak", "Palo Pinto", "Red River", "San Augustine", "San Jacinto", "San Patricio", "San Saba", "Tom Green", "Val Verde", "Van Zandt"]
for index, row in new_tx_cong.iterrows():
    if row["County"] in add:
        new_tx_cong.at[index, "County"] = row["County"]+ " County"

In [118]:
# create cname
new_tx_cong["CName"] = ""
for index, row in new_tx_cong.iterrows():
    new_tx_cong.at[index, "CName"] = "tx-" + ''.join(row["County"].split(" ")).lower() + "-" + row["Precinct #"].lower()

In [119]:
import difflib
def findSimilar(s1, lst):
    max_ = 0
    word = ""
    for s in lst:
        r = difflib.SequenceMatcher(None, s1, s).ratio()
        if r > max_:
            max_ = r
            word = s
    lst.remove(word)
    return word, lst

In [120]:
gdfCnames = list(gdf["CName"].unique())
newCnames = list(new_tx_cong["CName"].unique())
gdfCnames.sort()
newCnames.sort()

In [121]:
gdfCnamesLst = []
for cname in gdfCnames:
    if cname not in newCnames:
        gdfCnamesLst.append(cname)

In [122]:
newCnamesLst = []
for cname in newCnames:
    if cname not in gdfCnames:
        newCnamesLst.append(cname)

In [123]:
cnameMapper = {}
for cname in newCnamesLst:
    best, gdfCnamesLst = findSimilar(cname, gdfCnamesLst)
    cnameMapper[cname] = best
    print(cname, best)

tx-angelina-10a tx-angelina-0010a
tx-angelina-10b tx-angelina-0010b
tx-angelina-11b tx-angelina-011b
tx-angelina-17b tx-angelina-017b
tx-angelina-8a tx-angelina-0008a
tx-angelina-8b tx-angelina-0008b
tx-aransas-1a tx-aransas-001a
tx-aransas-4a tx-aransas-004a
tx-bailey-4a tx-bailey-0004a
tx-bailey-4b tx-bailey-0004b
tx-bell-404a tx-bell-0404a
tx-bell-404b tx-bell-0404b
tx-borden-1a tx-borden-001a
tx-borden-1b tx-borden-001b
tx-borden-2a tx-borden-002a
tx-borden-2b tx-borden-002b
tx-borden-3a tx-borden-003a
tx-borden-3b tx-borden-003b
tx-bowie-1a tx-bowie-001a
tx-bowie-1b tx-bowie-001b
tx-bowie-2a tx-bowie-002a
tx-bowie-2b tx-bowie-002b
tx-bowie-2c tx-bowie-002c
tx-bowie-2d tx-bowie-002d
tx-bowie-3a tx-bowie-003a
tx-bowie-3b tx-bowie-003b
tx-bowie-4a tx-bowie-004a
tx-bowie-4b tx-bowie-004b
tx-bowie-4c tx-bowie-004c
tx-bowie-4d tx-bowie-004d
tx-bowie-4e tx-bowie-004e
tx-bowie-5a tx-bowie-005a
tx-bowie-5b tx-bowie-005b
tx-bowie-5c tx-bowie-005c
tx-bowie-6a tx-bowie-006a
tx-bowie-6b tx-bow

tx-rockwall-3d tx-rockwall-003d
tx-rockwall-4a tx-rockwall-004a
tx-rockwall-4b tx-rockwall-004b
tx-rockwall-4c tx-rockwall-004c
tx-runnels-4a tx-runnels-0004a
tx-runnels-4b tx-runnels-0004b
tx-rusk-314a tx-rusk-0314a
tx-rusk-314b tx-rusk-0314b
tx-sansabacounty-2a tx-sansabacounty-002a
tx-sansabacounty-2b tx-sansabacounty-002b
tx-sansabacounty-3a tx-sansabacounty-003a
tx-sansabacounty-3b tx-sansabacounty-003b
tx-sansabacounty-4a tx-sansabacounty-004a
tx-sansabacounty-4b tx-sansabacounty-004b
tx-shackelford-406a tx-shackelford-0406a
tx-shackelford-406b tx-shackelford-0406b
tx-shelby-1n tx-shelby-001n
tx-shelby-1s tx-shelby-001s
tx-shelby-1se tx-shelby-01se
tx-shelby-1sw tx-shelby-01sw
tx-shelby-1w tx-shelby-001w
tx-shelby-9n tx-shelby-009n
tx-shelby-9s tx-shelby-009s
tx-stephens-1-11 tx-stephens-11-apr
tx-stephens-2-11 tx-stephens-11-feb
tx-stephens-3-11 tx-stephens-11-jan
tx-stephens-3-19 tx-stephens-19-apr
tx-stephens-4-11 tx-stephens-11-mar
tx-stephens-4-19 tx-stephens-19-mar
tx-steph

In [124]:
cnameMapper["tx-stephens-1-11"] = "tx-stephens-11-jan"
cnameMapper["tx-stephens-2-11"] = "tx-stephens-11-feb"
cnameMapper["tx-stephens-3-11"] = "tx-stephens-11-mar"
cnameMapper["tx-stephens-3-19"] = "tx-stephens-19-mar"
cnameMapper["tx-stephens-4-11"] = "tx-stephens-11-apr"
cnameMapper["tx-stephens-4-19"] = "tx-stephens-19-apr"
cnameMapper["tx-stephens-5-11"] = "tx-stephens-11-may"

cnameMapper["tx-wood-1-1"] = "tx-wood-1-jan"
cnameMapper["tx-wood-1-2"] = "tx-wood-2-jan"
cnameMapper["tx-wood-1-3"] = "tx-wood-3-jan"
cnameMapper["tx-wood-1-4"] = "tx-wood-4-jan"
cnameMapper["tx-wood-2-1"] = "tx-wood-1-feb"
cnameMapper["tx-wood-2-2"] = "tx-wood-2-feb"
cnameMapper["tx-wood-2-3"] = "tx-wood-3-feb"
cnameMapper["tx-wood-3-1"] = "tx-wood-1-mar"
cnameMapper["tx-wood-3-2"] = "tx-wood-2-mar"
cnameMapper["tx-wood-3-3"] = "tx-wood-3-mar"
cnameMapper["tx-wood-4-1"] = "tx-wood-1-apr"
cnameMapper["tx-wood-4-2"] = "tx-wood-2-apr"
cnameMapper["tx-wood-4-3"] = "tx-wood-3-apr"
cnameMapper["tx-wood-4-4"] = "tx-wood-4-apr"

In [125]:
# fix cname
for index, row in new_tx_cong.iterrows():
    if row["CName"] in cnameMapper.keys():
        new_tx_cong.at[index, "CName"] = cnameMapper[row["CName"]]

In [126]:
c = list(gdf.columns)
c.remove("CName")
c.remove("Precinct #")
c.remove("County")
gdf.drop(columns=c, inplace=True, axis=1)
gdf.head(1)

,County,Precinct #,CName
2865,Anderson,1,tx-anderson-1


In [127]:
merged = new_tx_cong.merge(gdf, on="CName", how="right")

In [128]:
merged.head()

,dist,County_x,Precinct #_x,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,County_y,Precinct #_y
0,1,Angelina,1,106.0,162.0,3.0,0.0,0.0,271.0,tx-angelina-1,Angelina,1
1,1,Angelina,10A,261.0,352.0,10.0,0.0,0.0,623.0,tx-angelina-0010a,Angelina,0010A
2,1,Angelina,10B,31.0,43.0,1.0,0.0,0.0,75.0,tx-angelina-0010b,Angelina,0010B
3,1,Angelina,11,115.0,1165.0,17.0,0.0,1.0,1298.0,tx-angelina-11,Angelina,11
4,1,Angelina,11B,104.0,561.0,17.0,0.0,1.0,683.0,tx-angelina-011b,Angelina,011B


In [130]:
merged.isna().sum()

dist            227
County_x        227
Precinct #_x    227
Cong16Dem       227
Cong16Rep       227
Cong16Lib       227
Cong16Grn       227
Cong16Other     227
Cong16Total     227
CName             0
County_y          0
Precinct #_y      0
dtype: int64

In [131]:
merged.drop(["County_x", "Precinct #_x"], axis=1, inplace=True)
merged.head(1)

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,County_y,Precinct #_y
0,1,106.0,162.0,3.0,0.0,0.0,271.0,tx-angelina-1,Angelina,1


In [150]:
len(gdf)

9082

In [151]:
len(new_tx_cong)

8858

In [152]:
len(merged)

9082

In [135]:
gdfCnames = list(gdf["CName"].unique())
newCnames = list(merged["CName"].unique())
gdfCnames.sort()
newCnames.sort()

for cname in newCnames:
    if cname not in gdfCnames:
        print(cname)

In [136]:
# there is duplicate in merged cname
dupCnames = list(merged[merged.duplicated(["CName"])]["CName"])

In [137]:
merged[merged["CName"] == dupCnames[0]]

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,County_y,Precinct #_y
3789,21,1538.0,2359.0,194.0,122.0,0.0,4213.0,tx-hays-224,Hays,224
3790,35,0.0,0.0,0.0,0.0,0.0,0.0,tx-hays-224,Hays,224


In [138]:
merged.drop(index=3790, axis=0, inplace=True)

In [139]:
merged[merged["CName"] == dupCnames[1]]

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,County_y,Precinct #_y
3798,21,0.0,0.0,0.0,0.0,0.0,0.0,tx-hays-335,Hays,335
3799,25,780.0,1271.0,97.0,0.0,0.0,2148.0,tx-hays-335,Hays,335


In [140]:
merged.drop(index=3798, axis=0, inplace=True)

In [141]:
merged[merged["CName"] == dupCnames[2]]

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,County_y,Precinct #_y
4670,25,1518.0,763.0,107.0,0.0,0.0,2388.0,tx-bell-405,Bell,405
4671,31,0.0,0.0,0.0,0.0,0.0,0.0,tx-bell-405,Bell,405


In [142]:
merged.drop(index=4071, axis=0, inplace=True)

In [149]:
merged.isna().sum()

dist            0
Cong16Dem       0
Cong16Rep       0
Cong16Lib       0
Cong16Grn       0
Cong16Other     0
Cong16Total     0
CName           0
County_y        0
Precinct #_y    0
dtype: int64

In [146]:
for index, row in merged.iterrows():
    if pd.isna(row["dist"]):
        merged.at[index, "dist"] = 8

In [148]:
merged.fillna(value=0, inplace=True)

In [155]:
merged.to_csv("TX_Congressional_2016_Cleaned.csv", index=False)

# 2018 congression

In [156]:
df = pd.read_csv("TX_2018_Congressional.csv")
df.head(1)

,County,FIPS,VTD,cntyvtd,Office,Name,Party,Incumbent,Votes
0,Anderson,1,0001,10001,U.S. Sen,Cruz,R,Y,659


In [157]:
tx_cong_vote = df[df["Office"].str.contains("U.S. Rep")]

# clean tx_cong_vote

In [158]:
rename = {"Office":"dist", "VTD":"Precinct #"}
tx_cong_vote.rename(rename, axis=1, inplace=True)
tx_cong_vote.drop(columns=["FIPS", "Incumbent"], axis=1, inplace=True)
tx_cong_vote.head(1)

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
66,Anderson,0001,10001,U.S. Rep 5,Gooden,R,672


In [159]:
def normalizeDist(df):
    for index, row in df.iterrows():
        d = row["dist"]
        d = d.split(" ")[-1]
        df.at[index, "dist"] = d

In [160]:
normalizeDist(tx_cong_vote)

In [161]:
def normalizePrecNum(df):
    for index, row in df.iterrows():
        df.at[index, "Precinct #"] = row["Precinct #"].lstrip("0")
normalizePrecNum(tx_cong_vote)


In [162]:
tx_cong_vote.sort_values(by=["dist", "Precinct #"], inplace=True)
tx_cong_vote.head(1)

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
1298,Angelina,1,50001,1,Gohmert,R,129


In [163]:
grouped = tx_cong_vote.groupby(["dist", "County", "Precinct #", "Name", "Party"])
grouped.sum()

Votes
dist County   Precinct # Name      Party       
1    Angelina 1          Callaway  L          3
                         Gohmert   R        129
                         McKellar  D         86
              10A        Callaway  L          3
                         Gohmert   R        229
...                                         ...
9    Harris   994        Rogers    I          2
              995        Green     D        819
                         Hernandez I         27
                         Kurtz     L         18
                         Rogers    I         33

[26785 rows x 1 columns]

In [164]:
tx_cong_vote["Party"].unique()

array(['R', 'D', 'L', 'I', 'W'], dtype=object)

In [178]:
# new dataframe
columns = ["dist", "County", "Precinct #", "Cong18Dem",
           "Cong18Rep", "Cong18Lib", "Cong18Grn",
           "Cong18Other", "Cong18Total"]
new_tx_cong = pd.DataFrame(columns=columns)
new_tx_cong.head()

,dist,County,Precinct #,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Grn,Cong18Other,Cong18Total


In [179]:
vote_column_name = {'D': "Cong18Dem", 'L': "Cong18Lib", 'I': "Cong18Indep", 'R': "Cong18Rep", 'W': "Cong18Other"}


# iterate the groupby df
print("################## Merging TX Dataframe ##################")

new_row = ""
cur_precinct = ""
total_votes = 0

for index, group in grouped:
    cur_row = group.loc[:].head(1)
    precinct = cur_row["Precinct #"].values[0]

    if precinct != cur_precinct:
        if cur_precinct != "":
            # add TotalVotes to new row
            new_row["Cong18Total"] = total_votes
            # add new_row to new_RI dataframe
            new_tx_cong = pd.concat([new_tx_cong, new_row], ignore_index=True, sort=False)
            
        # reset vars
        cur_precinct = precinct
        new_row = ""
        total_votes = 0
    
    if type(new_row) == str:
        # get columns other than Total (Vote)
        new_row = cur_row[["County", "dist", "Precinct #"]]

    candidate_key = cur_row["Party"].values[0]
    new_row[vote_column_name[candidate_key]] = cur_row["Votes"].values[0]
    total_votes += cur_row["Votes"].values[0]
    
print("\n################ Done Merging TX Dataframe ###############")

################## Merging TX Dataframe ##################

################ Done Merging TX Dataframe ###############


In [180]:
new_tx_cong.head()

,dist,County,Precinct #,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Grn,Cong18Other,Cong18Total,Cong18Indep
0,1,Angelina,1,86,129,3,NaN,NaN,218,NaN
1,1,Angelina,10A,193,229,3,NaN,NaN,425,NaN
2,1,Angelina,10B,24,28,0,NaN,NaN,52,NaN
3,1,Angelina,11,94,1095,11,NaN,NaN,1200,NaN
4,1,Angelina,11B,90,466,6,NaN,NaN,562,NaN


In [181]:
new_tx_cong.isna().sum()

dist              0
County            0
Precinct #        0
Cong18Dem         0
Cong18Rep       963
Cong18Lib      1469
Cong18Grn      9087
Cong18Other    8268
Cong18Total       0
Cong18Indep    8105
dtype: int64

In [182]:
new_tx_cong.drop(columns=["Cong18Grn"], inplace=True, axis=1)

In [183]:
a = sorted(list(tx_cong_vote["Precinct #"].unique()))

In [184]:
b = sorted(list(new_tx_cong["Precinct #"].unique()))

In [185]:
for tup in list(zip(a, b)):
    if tup[0] != tup[1]:
        print(tup)

('995', '996')
('996', '997')
('997', '998')
('998', '999')
('999', '9N')
('9N', '9S')


In [186]:
# new missing 996
tx_cong_vote[tx_cong_vote["Precinct #"] == "995"]

,County,Precinct #,cntyvtd,dist,Name,Party,Votes
175399,Harris,995,2010995,9,Green,D,819
176428,Harris,995,2010995,9,Kurtz,L,18
177218,Harris,995,2010995,9,Hernandez,I,27
177931,Harris,995,2010995,9,Rogers,I,33


In [187]:
new_tx_cong = new_tx_cong.append({"County":"Harris", "Precinct #":"995", "Cong18Lib":18, "Cong18Dem":819}, 
                                 ignore_index=True)
new_tx_cong

,dist,County,Precinct #,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep
0,1,Angelina,1,86,129,3,NaN,218,NaN
1,1,Angelina,10A,193,229,3,NaN,425,NaN
2,1,Angelina,10B,24,28,0,NaN,52,NaN
3,1,Angelina,11,94,1095,11,NaN,1200,NaN
4,1,Angelina,11B,90,466,6,NaN,562,NaN
...,...,...,...,...,...,...,...,...,...
9083,9,Harris,984,82,NaN,2,NaN,93,3.0
9084,9,Harris,985,126,NaN,21,NaN,166,5.0
9085,9,Harris,986,135,NaN,3,NaN,154,9.0
9086,9,Harris,994,70,NaN,5,NaN,83,2.0


In [188]:
new_tx_cong.at[9087, "dist"] = "9"
new_tx_cong.at[9087, "Cong18Indep"] = 60
new_tx_cong.at[9087, "Cong18Total"] = 897

In [189]:
new_tx_cong.isna().sum()

dist              0
County            0
Precinct #        0
Cong18Dem         0
Cong18Rep       964
Cong18Lib      1469
Cong18Other    8269
Cong18Total       0
Cong18Indep    8105
dtype: int64

In [190]:
# fill in nulls
new_tx_cong.fillna(value=0, inplace=True)
new_tx_cong.isna().sum()

dist           0
County         0
Precinct #     0
Cong18Dem      0
Cong18Rep      0
Cong18Lib      0
Cong18Other    0
Cong18Total    0
Cong18Indep    0
dtype: int64

In [197]:
add = ["Fort Bend", "De Witt", "El Paso", "Jeff Davis", "Jim Hogg", "Jim Wells", "La Salle", "Live Oak", "Palo Pinto", "Red River", "San Augustine", "San Jacinto", "San Patricio", "San Saba", "Tom Green", "Val Verde", "Van Zandt"]
for index, row in new_tx_cong.iterrows():
    if row["County"] in add:
        new_tx_cong.at[index, "County"] = row["County"]+ " County"

In [198]:
# create cname
new_tx_cong["CName"] = ""
for index, row in new_tx_cong.iterrows():
    new_tx_cong.at[index, "CName"] = "tx-" + ''.join(row["County"].split(" ")).lower() + "-" + row["Precinct #"].lower()

In [199]:
import difflib
def findSimilar(s1, lst):
    max_ = 0
    word = ""
    for s in lst:
        r = difflib.SequenceMatcher(None, s1, s).ratio()
        if r > max_:
            max_ = r
            word = s
    lst.remove(word)
    return word, lst

In [207]:
gdfCnames = list(gdf["CName"].unique())
newCnames = list(new_tx_cong["CName"].unique())
gdfCnames.sort()
newCnames.sort()

In [208]:
gdfCnamesLst = []
for cname in gdfCnames:
    if cname not in newCnames:
        gdfCnamesLst.append(cname)

In [209]:
newCnamesLst = []
for cname in newCnames:
    if cname not in gdfCnames:
        newCnamesLst.append(cname)

In [210]:
cnameMapper = {}
for cname in newCnamesLst:
    best, gdfCnamesLst = findSimilar(cname, gdfCnamesLst)
    cnameMapper[cname] = best
    print(cname, best)

In [204]:
cnameMapper["tx-stephens-1-11"] = "tx-stephens-11-jan"
cnameMapper["tx-stephens-2-11"] = "tx-stephens-11-feb"
cnameMapper["tx-stephens-3-11"] = "tx-stephens-11-mar"
cnameMapper["tx-stephens-3-19"] = "tx-stephens-19-mar"
cnameMapper["tx-stephens-4-11"] = "tx-stephens-11-apr"
cnameMapper["tx-stephens-4-19"] = "tx-stephens-19-apr"
cnameMapper["tx-stephens-5-11"] = "tx-stephens-11-may"

cnameMapper["tx-wood-1-1"] = "tx-wood-1-jan"
cnameMapper["tx-wood-1-2"] = "tx-wood-2-jan"
cnameMapper["tx-wood-1-3"] = "tx-wood-3-jan"
cnameMapper["tx-wood-1-4"] = "tx-wood-4-jan"
cnameMapper["tx-wood-2-1"] = "tx-wood-1-feb"
cnameMapper["tx-wood-2-2"] = "tx-wood-2-feb"
cnameMapper["tx-wood-2-3"] = "tx-wood-3-feb"
cnameMapper["tx-wood-3-1"] = "tx-wood-1-mar"
cnameMapper["tx-wood-3-2"] = "tx-wood-2-mar"
cnameMapper["tx-wood-3-3"] = "tx-wood-3-mar"
cnameMapper["tx-wood-4-1"] = "tx-wood-1-apr"
cnameMapper["tx-wood-4-2"] = "tx-wood-2-apr"
cnameMapper["tx-wood-4-3"] = "tx-wood-3-apr"
cnameMapper["tx-wood-4-4"] = "tx-wood-4-apr"

In [205]:
# fix cname
for index, row in new_tx_cong.iterrows():
    if row["CName"] in cnameMapper.keys():
        new_tx_cong.at[index, "CName"] = cnameMapper[row["CName"]]

In [206]:
c = list(gdf.columns)
c.remove("CName")
c.remove("Precinct #")
c.remove("County")
gdf.drop(columns=c, inplace=True, axis=1)
gdf.head(1)

,County,Precinct #,CName
2865,Anderson,1,tx-anderson-1


In [211]:
merged = new_tx_cong.merge(gdf, on="CName", how="right")

In [212]:
merged.head()

,dist,County_x,Precinct #_x,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
0,1,Angelina,1,86,129,3,0,218,0.0,tx-angelina-1,Angelina,1
1,1,Angelina,10A,193,229,3,0,425,0.0,tx-angelina-0010a,Angelina,0010A
2,1,Angelina,10B,24,28,0,0,52,0.0,tx-angelina-0010b,Angelina,0010B
3,1,Angelina,11,94,1095,11,0,1200,0.0,tx-angelina-11,Angelina,11
4,1,Angelina,11B,90,466,6,0,562,0.0,tx-angelina-011b,Angelina,011B


In [213]:
merged.isna().sum()

dist            0
County_x        0
Precinct #_x    0
Cong18Dem       0
Cong18Rep       0
Cong18Lib       0
Cong18Other     0
Cong18Total     0
Cong18Indep     0
CName           0
County_y        0
Precinct #_y    0
dtype: int64

In [214]:
merged.drop(["County_x", "Precinct #_x"], axis=1, inplace=True)
merged.head(1)

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
0,1,86,129,3,0,218,0.0,tx-angelina-1,Angelina,1


In [215]:
len(gdf)

9082

In [216]:
len(new_tx_cong)

9088

In [217]:
len(merged)

9088

In [222]:
gdfCnames = list(gdf["CName"].unique())
newCnames = list(merged["CName"].unique())
gdfCnames.sort()
newCnames.sort()

for cname in newCnames:
    if cname not in gdfCnames:
        print(cname)

In [223]:
# there is duplicate in merged cname
dupCnames = list(merged[merged.duplicated(["CName"])]["CName"])

In [224]:
merged[merged["CName"] == dupCnames[0]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
344,10,748,1764,56,0,2568,0.0,tx-harris-114,Harris,114
345,8,0,0,0,0,0,0.0,tx-harris-114,Harris,114


In [225]:
merged.drop(index=345, axis=0, inplace=True)

In [226]:
merged[merged["CName"] == dupCnames[1]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
3350,2,69,37,0,0,108,2.0,tx-harris-983,Harris,983
3351,8,0,0,0,0,0,0.0,tx-harris-983,Harris,983


In [229]:
merged.drop(index=3351, axis=0, inplace=True)

In [230]:
merged[merged["CName"] == dupCnames[2]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
3698,21,1125,1261,51,0,2437,0.0,tx-bexar-3121,Bexar,3121
3699,23,0,0,0,0,0,0.0,tx-bexar-3121,Bexar,3121


In [231]:
merged.drop(index=3699, axis=0, inplace=True)

In [232]:
merged[merged["CName"] == dupCnames[3]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
3791,21,2197,2152,126,0,4475,0.0,tx-hays-224,Hays,224
3792,35,0,0,0,0,0,0.0,tx-hays-224,Hays,224


In [233]:
merged.drop(index=3792, axis=0, inplace=True)

In [234]:
merged[merged["CName"] == dupCnames[4]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
3800,21,0,0,0,0,0,0.0,tx-hays-335,Hays,335
3801,25,914,1324,59,0,2297,0.0,tx-hays-335,Hays,335


In [235]:
merged.drop(index=3800, axis=0, inplace=True)

In [236]:
merged[merged["CName"] == dupCnames[5]]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Indep,CName,County_y,Precinct #_y
4673,25,1429,512,44,0,1985,0.0,tx-bell-405,Bell,405
4674,31,0,0,0,0,0,0.0,tx-bell-405,Bell,405


In [237]:
merged.drop(index=4674, axis=0, inplace=True)

In [239]:
merged.isna().sum()

dist            0
Cong18Dem       0
Cong18Rep       0
Cong18Lib       0
Cong18Other     0
Cong18Total     0
Cong18Indep     0
CName           0
County_y        0
Precinct #_y    0
dtype: int64

In [240]:
merged.to_csv("TX_Congressional_2018_Cleaned.csv", index=False)